In [36]:
import pandas as pd

In [37]:
nba_df = pd.read_csv("/Users/gauravmohan/Downloads/2022-2023_NBA_PbP_Logs/[10-18-2022]-[06-12-2023]-combined-stats.csv")

In [38]:
nba_df = nba_df.drop(index=0)
nba_df.head()

,game_id,data_set,date,a1,a2,a3,a4,a5,h1,h2,...,reason,result,steal,type,shot_distance,original_x,original_y,converted_x,converted_y,description
1,22200001,2022-23 Regular Season,2022-10-18,Joel Embiid,Tobias Harris,P.J. Tucker,Tyrese Maxey,James Harden,Al Horford,Jaylen Brown,...,NaN,NaN,NaN,jump ball,NaN,NaN,NaN,NaN,NaN,Jump Ball Horford vs. Embiid: Tip to Harris
2,22200001,2022-23 Regular Season,2022-10-18,Joel Embiid,Tobias Harris,P.J. Tucker,Tyrese Maxey,James Harden,Al Horford,Jaylen Brown,...,NaN,missed,NaN,turnaround fadeaway,13.0,-118.0,50.0,36.8,10.0,Brown BLOCK (1 BLK): MISS Embiid 13' Turnaroun...
3,22200001,2022-23 Regular Season,2022-10-18,Joel Embiid,Tobias Harris,P.J. Tucker,Tyrese Maxey,James Harden,Al Horford,Jaylen Brown,...,NaN,NaN,NaN,rebound offensive,NaN,NaN,NaN,NaN,NaN,Tucker REBOUND (Off:1 Def:0)
4,22200001,2022-23 Regular Season,2022-10-18,Joel Embiid,Tobias Harris,P.J. Tucker,Tyrese Maxey,James Harden,Al Horford,Jaylen Brown,...,out of bounds lost ball,NaN,NaN,out of bounds lost ball,NaN,NaN,NaN,NaN,NaN,Tucker Out of Bounds Lost Ball Turnover (P1.T1)
5,22200001,2022-23 Regular Season,2022-10-18,Joel Embiid,Tobias Harris,P.J. Tucker,Tyrese Maxey,James Harden,Al Horford,Jaylen Brown,...,NaN,made,NaN,driving floating bank jump shot,13.0,120.0,55.0,37.0,83.5,Smart 13' Driving Floating Bank Jump Shot (2 PTS)


### Classify Shots into Shot Zones

In [39]:
nba_df['converted_x'] = nba_df['converted_x'].astype('float')
nba_df['converted_y'] = nba_df['converted_y'].astype('float')

In [40]:
import pandas as pd
import math

# Define the UDF to calculate the shot distance from the relevant basket
def calculate_shot_distance(x, y):
    basket_1_x, basket_1_y = 25, 5.25
    basket_2_x, basket_2_y = 25, 94 - 5.25

    # Determine which side of the court the shot was taken
    if y <= 47:
        basket_x, basket_y = basket_1_x, basket_1_y
    else:
        basket_x, basket_y = basket_2_x, basket_2_y

    # Calculate distance from the relevant basket
    distance_from_basket = math.sqrt((x - basket_x) ** 2 + (y - basket_y) ** 2)
    return distance_from_basket

nba_df['shot_dist_calc'] = nba_df.apply(lambda row: calculate_shot_distance(row['converted_x'], row['converted_y']), axis=1)

In [41]:
import pandas as pd
import math

# Define the UDF to determine the shot zone, considering the full court
def determine_shot_zone(x, y, shot_distance):
    basket_1_x, basket_1_y = 25, 5.25
    basket_2_x, basket_2_y = 25, 94 - 5.25

    # Determine which side of the court the shot was taken
    if y <= 47:
        basket_x, basket_y = basket_1_x, basket_1_y
    else:
        basket_x, basket_y = basket_2_x, basket_2_y

    # Calculate distance from the relevant basket
    distance_from_basket = math.sqrt((x - basket_x) ** 2 + (y - basket_y) ** 2)

    # Determine zones
    if distance_from_basket <= 4:
        return "RESTRICTED AREA"
    elif abs(x - basket_x) <= 6 and distance_from_basket > 4 and distance_from_basket <= 15:
        return "IN THE PAINT (NON-RA)"
    elif distance_from_basket < 22 and (abs(x - basket_x) > 6 or distance_from_basket > 15):
        return "MID-RANGE"
    elif distance_from_basket >= 22 and (x < 14 or x > 44) and ((basket_y == 5.25 and y <= 10) or (basket_y == 88.75 and y >= 84)):
        return "CORNER 3"
    elif float(shot_distance) > 50 or (distance_from_basket > 22 and ((basket_y == 5.25 and y > 10) or (basket_y == 88.75 and y < 84))):
        return "ABOVE THE BREAK 3"
    else:
        return "Unknown Zone"


# Apply the UDF to the DataFrame
nba_df['shot_zone'] = nba_df.apply(lambda row: determine_shot_zone(row['converted_x'], row['converted_y'], row['shot_distance']), axis=1)

##### Impute Missing Shot Coordinates using PbP Descriptions

In [42]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

def classify_shot_with_gpt(description):
    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
            {"role": "system", "content": "You are an NBA analytics expert. Your task is to classify NBA play-by-play descriptions into shot zones."},
            {"role": "user", "content": f"Classify the following basketball shot description into one of these categories: 'MID-RANGE', 'IN THE PAINT (NON-RA)', 'RESTRICTED AREA', 'CORNER 3', 'ABOVE THE BREAK 3'.\n\nDescription: \"{description}\"\n\nShot Zone:"}
    ],
    n=1,
    max_tokens=8,
    temperature=0.5
    )
    classified_zone = completion.choices[0].message.content
    return str(classified_zone)

# Function to classify and update 'Unknown Zone' entries
def classify_and_update(df):
    for index, row in df.iterrows():
        if row['event_type'] == 'shot' and row['shot_zone'] == 'Unknown Zone':
            description = row['description']
            classified_zone = classify_shot_with_gpt(description)
            if "RESTRICTED AREA" in classified_zone:
                zone = "RESTRICTED AREA"
            elif "IN THE PAINT" in classified_zone:
                zone = "IN THE PAINT (NON-RA)"
            elif "MID-RANGE" in classified_zone:
                zone = "MID-RANGE"
            elif "CORNER" in classified_zone:
                zone = "CORNER 3"
            elif "ABOVE THE BREAK" in classified_zone:
                zone = "ABOVE THE BREAK 3"
            else:
                zone = "Unknown Zone"
            print(zone)
            df.at[index, 'shot_zone'] = zone
    
    return df

# Apply classification and update
nba_df = classify_and_update(nba_df)

ABOVE THE BREAK 3
ABOVE THE BREAK 3
MID-RANGE
IN THE PAINT (NON-RA)
ABOVE THE BREAK 3
Unknown Zone
Unknown Zone
ABOVE THE BREAK 3
ABOVE THE BREAK 3
IN THE PAINT (NON-RA)
IN THE PAINT (NON-RA)
ABOVE THE BREAK 3
RESTRICTED AREA
RESTRICTED AREA
RESTRICTED AREA
ABOVE THE BREAK 3
MID-RANGE
ABOVE THE BREAK 3
RESTRICTED AREA
Unknown Zone
ABOVE THE BREAK 3
ABOVE THE BREAK 3
ABOVE THE BREAK 3
RESTRICTED AREA
Unknown Zone
Unknown Zone
MID-RANGE
MID-RANGE
ABOVE THE BREAK 3
ABOVE THE BREAK 3
MID-RANGE
MID-RANGE
RESTRICTED AREA
MID-RANGE
ABOVE THE BREAK 3
RESTRICTED AREA
MID-RANGE
RESTRICTED AREA
ABOVE THE BREAK 3
MID-RANGE
RESTRICTED AREA
ABOVE THE BREAK 3
RESTRICTED AREA
ABOVE THE BREAK 3
ABOVE THE BREAK 3
ABOVE THE BREAK 3
ABOVE THE BREAK 3
RESTRICTED AREA
ABOVE THE BREAK 3
RESTRICTED AREA
RESTRICTED AREA
RESTRICTED AREA
MID-RANGE
IN THE PAINT (NON-RA)
Unknown Zone
RESTRICTED AREA
RESTRICTED AREA
ABOVE THE BREAK 3
ABOVE THE BREAK 3
MID-RANGE
MID-RANGE
ABOVE THE BREAK 3
Unknown Zone
IN THE PAINT 

#### Define States

In [44]:
import pandas as pd
import numpy as np

def find_previous_shot_info(index):
    for i in range(index-1, -1, -1):
        if nba_df.at[i, 'result'] in ['made', 'missed']:
            return nba_df.at[i, 'result'], nba_df.at[i, 'shot_zone']
    #return 'missed', 'RESTRICTED AREA'

# Apply the function to create new columns
prev_info = [find_previous_shot_info(i) if nba_df.at[i, 'type'] == 'shooting' else (None, None) for i in range(1,len(nba_df)+1)]
nba_df['prev_result'] = [x[0] for x in prev_info]
nba_df['prev_shot_zone'] = [x[1] for x in prev_info]

# Define conditions and corresponding state values
conditions = [
    (nba_df['event_type'] == 'turnover') | (nba_df['type'].isin(['offensive charge', 'offensive foul'])),
    (nba_df['event_type'] == 'jump ball'),
    (nba_df['event_type'] == 'timeout'),
    (nba_df['event_type'] == 'start of period'),
    (nba_df['event_type'] == 'substitution'),
    (nba_df['event_type'] == 'end of period'),
    (nba_df['type'].isin(['3pt jump shot', '3pt step back jump shot', '3pt running jump shot', '3pt fadeaway jumper', '3pt pullup jump shot', 
                          '3pt running pull-up jump shot', '3pt step back bank jump shot', '3pt turnaround jump shot', '3pt jump bank shot',
                          '3pt turnaround fadeaway'])),
    (nba_df['type'].isin(['dunk', 'driving floating bank jump shot', 'floating jump shot', 'cutting layup shot', 'jump shot', 
                          'layup', 'driving layup', 'fadeaway jumper', 'hook shot', 'driving dunk', 'driving floating jump shot', 
                          'driving reverse layup', 'jump bank shot', 'turnaround fadeaway', 'cutting finger roll layup shot', 
                          'hook bank shot', 'layup'])),
    (nba_df['type'] == 'rebound defensive'),
    (nba_df['type'].isin(['rebound offensive', 'team rebound'])),
    (nba_df['event_type'] == 'foul') & (nba_df['type'].isin(['personal', 'personal take'])), 
    ((nba_df['event_type'] == 'foul') & (nba_df['type'] == 'shooting') & (nba_df['prev_shot_zone'].isin(['RESTRICTED AREA', 'IN THE PAINT (NON-RA)', 'MID-RANGE'])) & (nba_df['prev_result'] == 'missed')) | (nba_df['type'].isin(['flagrant-1','flagrant-2'])),
    (nba_df['event_type'] == 'foul') & (nba_df['type'] == 'shooting') & (nba_df['prev_shot_zone'].isin(['ABOVE THE BREAK 3', 'CORNER 3'])) & (nba_df['prev_result'] == 'missed'),
    ((nba_df['event_type'] == 'foul') & (nba_df['type'] == 'shooting') & (nba_df['result'].shift(1) == 'made')) | (nba_df['type'] == 'free throw technical')
]

# Define corresponding state values for each condition
state_values = [
    'turnover',
    'jump ball',
    'timeout',
    'start of period', 
    'substitution',
    'end of period',
    '3pt shot',
    '2pt shot',
    'defensive rebound',
    'offensive rebound',
    'Non-shooting foul',
    '2-pt shooting foul',
    '3-pt shooting foul',
    '1-pt shooting foul'
]

# Apply conditions to create the 'state' column
nba_df['state'] = np.select(conditions, state_values, default='OTHER')
nba_df = nba_df[nba_df['state'] != 'OTHER']

In [46]:
nba_df.reset_index(drop=True, inplace=True)

In [49]:
# Step 1: Calculate state counts by team
state_counts = nba_df.groupby('team')['state'].value_counts()

# Step 2: Calculate total states per team
total_states_per_team = nba_df.groupby('team')['state'].size()

# Step 3: Calculate state proportions by team
state_proportions = state_counts / total_states_per_team

In [50]:
state_proportions = dict(state_proportions)

In [53]:
import json

# Function to load configuration from JSON file
def load_config(config_file):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

# Example usage:
shot_zone_dict = load_config('config.json')
ft_dict = load_config('config_ft.json')

In [70]:
nba_df[pd.isna(nba_df['team'])]['state'].unique()

array(['end of period', 'start of period'], dtype=object)

In [113]:
states

array(['jump ball', '2pt shot', 'offensive rebound', 'turnover',
       '3pt shot', 'defensive rebound', '3-pt shooting foul',
       'Non-shooting foul', '2-pt shooting foul', 'substitution',
       '1-pt shooting foul', 'timeout', 'end of period',
       'start of period'], dtype=object)

#### Calculate EPV

In [72]:
# Calculate EPV for each event
def calculate_epv(row, state_proportions, shot_zone_dict, ft_dict):
    team = row['team']
    shot_zone = row['shot_zone'].lower()
    if row['state'] in ['start of period', 'end of period']:
        return 0 
    for zone_dict in shot_zone_dict[team]:
        normalized_zone_dict = {key.lower(): value for key, value in zone_dict.items()}
        if shot_zone in normalized_zone_dict:
            shot_zone_pctg = float(normalized_zone_dict[shot_zone])/100
        elif shot_zone == 'unknown zone':
            shot_zone_pctg = 0
    team_ft_pct = ft_dict[team]

    if shot_zone in ['CORNER 3', 'ABOVE THE BREAK 3']:
        epv = (
            3 * shot_zone_pctg +
            3 * state_proportions[(team, '3-pt shooting foul')] * team_ft_pct +
            2 * state_proportions[(team,'2-pt shooting foul')] * team_ft_pct +
            1 * state_proportions[(team,'1-pt shooting foul')] * team_ft_pct
        )
    else:
        epv = (
            2 * shot_zone_pctg +
            3 * state_proportions[(team, '3-pt shooting foul')] * team_ft_pct +
            2 * state_proportions[(team,'2-pt shooting foul')] * team_ft_pct +
            1 * state_proportions[(team,'1-pt shooting foul')] * team_ft_pct
        )
    
    return epv

# Calculate EPV for each row
nba_df['EPV'] = nba_df.apply(calculate_epv, axis=1, state_proportions=state_proportions, shot_zone_dict=shot_zone_dict, ft_dict=ft_dict)

In [74]:
# Shift the 'state' column by one row to get the next state for each event
nba_df['next_state'] = nba_df['state'].shift(-1)
nba_df['previous_state'] = nba_df['state'].shift(1)
# For the last event in each game, set the next state to an appropriate value (e.g., 'end of period')
nba_df.loc[nba_df['game_id'] != nba_df['game_id'].shift(-1), 'next_state'] = 'end of period'

# Drop rows where 'next_state' is NaN (typically the last event in the dataset)
nba_df = nba_df.dropna(subset=['next_state'])


In [92]:
nba_df.to_csv('nba_pbp_states.csv')

#### Generate Transition Matrices By Team

In [ ]:
nba_df = pd.read_csv('nba_pbp_states.csv', index_col=0)

In [109]:
import pandas as pd
import numpy as np

regular_season_df = nba_df[nba_df['data_set'] == '2022-23 Regular Season']

# Get unique states
states = nba_df['state'].unique()

# Initialize a dictionary to hold transition matrices for each team
team_transition_matrices = {}

# Group by the 'team' column
grouped = regular_season_df.groupby('team')

# Iterate over each team and calculate the transition matrix
for team, group in grouped:
    # Initialize the transition matrix for this team
    transition_matrix = pd.DataFrame(np.zeros((len(states), len(states))), index=states, columns=states)
    
    # Calculate transition counts
    for i in range(1, len(group)):  # Start from 1 to ensure there is a previous state
        current_state = group.iloc[i]['state']
        previous_state = group.iloc[i]['previous_state']
        # Check if both states are valid
        if pd.notna(previous_state) and pd.notna(current_state):
            transition_matrix.at[previous_state, current_state] += 1
    
    # Count 'start of period' as a valid state in state_counts
    state_counts = group['state'].value_counts()
    start_of_period_counts = group['previous_state'].value_counts()
    if 'start of period' not in state_counts:
        state_counts['start of period'] = 0
    for state in start_of_period_counts.index:
        if state != 'end of period':
            state_counts[state] += start_of_period_counts[state]

    # Normalize to get transition probabilities
    for state in states:
        if state == 'end of period':
            transition_matrix.at[state, 'start of period'] = 1
        else:
            if state in state_counts:
                transition_matrix.loc[state] = transition_matrix.loc[state] / state_counts[state]
    
    # Fill NaN values with 0
    transition_matrix.fillna(0, inplace=True)
    
    # Store the transition matrix in the dictionary
    team_transition_matrices[team] = transition_matrix

# Now, team_transition_matrices contains the transition matrix for each team


In [111]:
### Store the file

import pickle

# Assuming you have already populated team_transition_matrices dictionary

# Define a filename for your saved file
filename = 'team_transition_matrices.pkl'

# Save the dictionary using pickle
with open(filename, 'wb') as file:
    pickle.dump(team_transition_matrices, file)

print(f"Saved team transition matrices to {filename}")


Saved team transition matrices to team_transition_matrices.pkl
